In [1]:

from lxml.html import fromstring
import requests
import pandas as pd
from IPython.display import Markdown

#INICIA FIREBASE
import firebase_admin
from firebase_admin import firestore

cred_obj = firebase_admin.credentials.Certificate('./apprenova-c3758-firebase-adminsdk-99epx-89d70a8235.json')
default_app = firebase_admin.initialize_app(cred_obj, {
    'databaseURL': '/'
})
db = firestore.client()
ref = db.collection(u'2008').document('xxCwfvzXXT3nuCqUpaJq')


#LISTA DE CIDADES, NUMERO IDENTIFICADOR E URL DE ACESSO.
def consulta():
    url = 'https://resultados.tre-rs.jus.br/eleicoes/2008/1turno/'
    req = requests.get(url)
    root = fromstring(req.text)
    ul = root.xpath('.//ul[@id="main_ul"]')[0]
    cities = []
    for li in ul.xpath('.//li')[0:]:
        cities.append({
            'name': li.xpath('.//a')[0].text_content(),
            'href': li.xpath('.//a')[0].attrib['href'],
            'url': url + li.xpath('.//a')[0].attrib['href']
        })

    # IMPRIMIR EM FORMATO DE TABELA AS INFORMAÇÕES DAS CIDADES
    #display(pd.DataFrame(cities)) 

    # VARRE TODAS AS CIDADES E VERIFICA OS ELEITOS
    data = []
    for index, city in enumerate(cities):
        url = cities[index]["url"]
        req = requests.get(url)
        root = fromstring(req.text)
        tables = root.xpath('.//table[@summary="Vota��o no munic�pio"]')
        h2_s = root.xpath('.//h2')
        eleitos_class = ["lr_even eleito ", "eleito "]
        city = cities[index]["name"]
        id = cities[index]["href"]
        for index,table in enumerate(tables):
            data.append({
                'id': id,
                'city': city,
                'year': "2008",
                'turno': "PRIMEIRO TURNO",
                'title': h2_s[index].text_content(),
                'values': []
            })
            for classe in eleitos_class:
                for tr in table.xpath(f'.//tr[@class="{classe}"]'):
                    data[-1]['values'].append({ 
                    'name': tr.xpath('.//a')[0].text_content(),
                    'partido': tr.xpath('.//td')[3].text_content(),
                    'votação': tr.xpath('.//td')[4].text_content()
                })
                
        
        # IMPRIMIR AS INFORMAÇÕES EM FORMATO DE TABELA
        # for d in data:
        #     display(Markdown(f"# {d['city']} - PRIMEIRO TURNO"))
        #     display(Markdown(f"## {d['title']}"))
        #     display(pd.DataFrame(d['values']))
            
    #VARRE O SEGUNDO TURNO
    url2 = 'https://resultados.tre-rs.jus.br/eleicoes/2008/2turno/'
    req2 = requests.get(url2)
    root = fromstring(req2.text)
    ul = root.xpath('.//ul[@id="main_ul"]')[0]
    cities2 = []
    for li in ul.xpath('.//li')[0:]:
        cities2.append({
            'name': li.xpath('.//a')[0].text_content(),
            'href': li.xpath('.//a')[0].attrib['href'],
            'url2': url2 + li.xpath('.//a')[0].attrib['href']
        })
    for idCity, city2 in enumerate(cities2):
        req = requests.get(city2["url2"])
        root = fromstring(req.text)
        tables = root.xpath('.//table[@summary="Vota��o no munic�pio"]')
        h2_s = root.xpath('.//h2')
        eleitos_class = ["lr_even eleito ", "eleito "]
        for index,table in enumerate(tables):
            data.append({
                'id': city2["href"],
                'city': city2["name"],
                'year': "2008",
                'turno': "SEGUNDO TURNO",
                'title': h2_s[index].text_content(),
                'values': []
            })
            for classe in eleitos_class:
                for tr in table.xpath(f'.//tr[@class="{classe}"]'):
                    data[-1]['values'].append({ 
                    'name': tr.xpath('.//a')[0].text_content(),
                    'partido': tr.xpath('.//td')[3].text_content(),
                    'votação': tr.xpath('.//td')[4].text_content()
                })
                    
        # IMPRIMIR AS INFORMAÇÕES EM FORMATO DE TABELA
        # for d in data:
        #     display(Markdown(f"# {d['city']} - SEGUNDO TURNO"))
        #     display(Markdown(f"## {d['title']}"))
        #     display(pd.DataFrame(d['values']))
    
    for d in data:
        if d['title'] == 'Prefeito' and d['turno'] == 'SEGUNDO TURNO':
            for i in data:
                if i['title'] == 'Prefeito' and i['turno'] == 'PRIMEIRO TURNO' and i['city'] == d['city']:
                    data.remove(i)
                    
    for eleito in data:
        if eleito['title'] == 'Prefeito':
            ref.set({
                'Prefeito': eleito
            }, merge=True)
        if eleito['title'] == 'Vereador':
            ref.set({
                'Vereador': eleito
            }, merge=True)

    #REGISTRA OD DADOS NO FIREBASE, MERGE TRUE NÃO SUBSTITUI O CONTEÚDO QUE JÁ ESTÁ NA COLEÇÃO
    # ref.set({
    #     '2008': data
    # }, merge=True)
    return data

consulta()

[{'id': 'RS88986.html',
  'city': 'ACEGU�',
  'year': '2008',
  'turno': 'PRIMEIRO TURNO',
  'title': 'Prefeito',
  'values': [{'name': 'GERHARD MARTENS',
    'partido': 'PSDB',
    'votação': '1625'}]},
 {'id': 'RS88986.html',
  'city': 'ACEGU�',
  'year': '2008',
  'turno': 'PRIMEIRO TURNO',
  'title': 'Vereador',
  'values': [{'name': 'ALFREDO CASTILLOS DE LOS SANTOS',
    'partido': 'PSDB',
    'votação': '174'},
   {'name': 'PEDRO ANILDO ARDENGHI', 'partido': 'PT', 'votação': '148'},
   {'name': 'LIZIANE BRASIL JARDIM', 'partido': 'PMDB', 'votação': '125'},
   {'name': 'REOVALDO RODRIGUES', 'partido': 'PT', 'votação': '109'},
   {'name': 'ODETE DA SILVA RIBEIRO', 'partido': 'PT', 'votação': '191'},
   {'name': 'NICKOLAI PENNER', 'partido': 'DEM', 'votação': '156'},
   {'name': 'EDMUNDO PICHLER', 'partido': 'PMDB', 'votação': '145'},
   {'name': 'WILLI JANZEN', 'partido': 'DEM', 'votação': '127'},
   {'name': 'JORGE ROBERTO VAZ', 'partido': 'PMDB', 'votação': '110'}]},
 {'id': 'RS8